In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-earthdata')

In [2]:
import ee
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, mapping
import datetime

# Initialize the Earth Engine API
ee.Initialize()

# Define the geometry for contiguous USA
usa_coords = [
    [-125.1803892906456, 35.26328285844432],
    [-117.08916345892665, 33.2311514593429],
    [-114.35640058749676, 32.92199940444295],
    [-110.88773544819885, 31.612036247094473],
    [-108.91086200144109, 31.7082477979397],
    [-106.80030780089378, 32.42079476218232],
    [-103.63413436750255, 29.786401496314422],
    [-101.87558377066483, 30.622527701868453],
    [-99.40039768482492, 28.04018292597704],
    [-98.69085295525215, 26.724810345780593],
    [-96.42355704777482, 26.216515704595633],
    [-80.68508661702214, 24.546812350183075],
    [-75.56173032587596, 26.814533788629998],
    [-67.1540159827795, 44.40095539443753],
    [-68.07548734644243, 46.981170472447374],
    [-69.17500995805074, 46.98158998130476],
    [-70.7598785138901, 44.87172183866657],
    [-74.84994741250935, 44.748084983808],
    [-77.62168256782745, 43.005725611950055],
    [-82.45987924104175, 41.41068867019324],
    [-83.38318501671864, 42.09979904377044],
    [-82.5905167831457, 45.06163491639556],
    [-84.83301910769038, 46.83552648258547],
    [-88.26350848510909, 48.143646480291835],
    [-90.06706251069104, 47.553445811024204],
    [-95.03745451438925, 48.9881557770297],
    [-98.45773319567587, 48.94699366043251],
    [-101.7018751401119, 48.98284560308372],
    [-108.43164852530356, 48.81973606668503],
    [-115.07339190755627, 48.93699058308441],
    [-121.82530604190744, 48.9830983403776],
    [-122.22085227110232, 48.63535795404536],
    [-124.59504332589562, 47.695726563030405],
    [-125.1803892906456, 35.26328285844432]
]

def create_usa_geometry():
    """Create an Earth Engine geometry object for the contiguous USA."""
    return ee.Geometry.Polygon([usa_coords])

def compute_area(feature):
    """Compute the area of a feature and set it as a property."""
    return feature.set({'area': feature.area()})

def compute_centroid(feature):
    """Compute the centroid coordinates of a feature and set them as properties."""
    centroid = feature.geometry().centroid().coordinates()
    return feature.set({
        'lon': centroid.get(0),
        'lat': centroid.get(1)
    })

def compute_date(feature):
    """Set start and end dates as properties of a feature."""
    return feature.set({
        'start_date': ee.Date(feature.get('IDate')),
        'end_date': ee.Date(feature.get('FDate'))
    })

def ee_array_to_df(arr, list_of_bands):
    """Convert Earth Engine array to pandas DataFrame."""
    df = pd.DataFrame(arr)
    
    # Select subset of columns if list_of_bands is not empty
    if list_of_bands:
        df = df[list_of_bands]
    
    return df

def ee_featurecollection_to_gdf(fc):
    """Convert Earth Engine FeatureCollection to GeoPandas DataFrame."""
    features = fc.getInfo()['features']
    
    # Extract the geometry and properties from each feature
    geometries = []
    properties = []
    
    for feature in features:
        # Convert GEE geometry to Shapely geometry
        geom = feature['geometry']
        if geom['type'] == 'Polygon':
            geometry = Polygon(geom['coordinates'][0])
        else:
            # Handle other geometry types if needed
            continue
            
        geometries.append(geometry)
        properties.append(feature['properties'])
    
    # Create GeoDataFrame
    df = pd.DataFrame(properties)
    gdf = gpd.GeoDataFrame(df, geometry=geometries, crs="EPSG:4326")
    
    # Convert timestamps to datetime
    if 'IDate' in gdf.columns:
        gdf['start_date'] = pd.to_datetime(gdf['IDate'].min(), unit='ms')
        gdf['end_date'] = pd.to_datetime(gdf['IDate'].max(), unit='ms')
    
    return gdf

def get_fires(year, min_size=1e7):
    """
    Get fires from the GlobFire database for a specific year and minimum size.
    
    Args:
        year (str): The year to filter fires for
        min_size (float): Minimum fire size in square meters (default: 1e7)
    
    Returns:
        geopandas.GeoDataFrame: GeoDataFrame containing fire data
    """
    # Create geometry
    geometry = create_usa_geometry()
    
    # Create date range for the year
    start_date = ee.Date(f'{year}-01-01')
    end_date = ee.Date(f'{year}-12-31')
    
    # Get and filter fire polygons
    polygons = (ee.FeatureCollection('JRC/GWIS/GlobFire/v2/FinalPerimeters')
                .filter(ee.Filter.gt('IDate', start_date.millis()))
                .filter(ee.Filter.lt('IDate', end_date.millis()))
                .filterBounds(geometry))
    
    # Apply area calculations and filters
    polygons = polygons.map(compute_area)
    polygons = (polygons
                .filter(ee.Filter.gt('area', min_size))
                .filter(ee.Filter.lt('area', 1e20)))
    
    # Compute additional properties
    polygons = polygons.map(compute_centroid).map(compute_date)
    
    # Convert to GeoDataFrame
    gdf = ee_featurecollection_to_gdf(polygons)
    
    return gdf

if __name__ == "__main__":
    # Example usage
    YEAR = "2018"
    MIN_SIZE = 1e7  # 1 square kilometers
    
    # Get fires as a GeoDataFrame
    fires_gdf = get_fires(YEAR, MIN_SIZE)
    
    print(f"Retrieved {len(fires_gdf)} fires for {YEAR}")
    print("\nFirst few rows:")
    print(fires_gdf.head())
    
    # Example: Basic statistics
    print("\nArea statistics (square meters):")
    print(fires_gdf['area'].describe())
    
    # # Example: Save to file
    # output_file = f"us_fires_{YEAR}.gpkg"
    # fires_gdf.to_file(output_file, driver="GPKG")
    # print(f"\nSaved to {output_file}")

: 

#### Download GlobFire Daily Fire Event Detection Based on MCD64A1 using Google Earth Engine

In [ ]:
import ee
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, mapping
import datetime

# Initialize the Earth Engine API
ee.Initialize()

# Define the geometry for contiguous USA
usa_coords = [
    [-125.1803892906456, 35.26328285844432],
    [-117.08916345892665, 33.2311514593429],
    [-114.35640058749676, 32.92199940444295],
    [-110.88773544819885, 31.612036247094473],
    [-108.91086200144109, 31.7082477979397],
    [-106.80030780089378, 32.42079476218232],
    [-103.63413436750255, 29.786401496314422],
    [-101.87558377066483, 30.622527701868453],
    [-99.40039768482492, 28.04018292597704],
    [-98.69085295525215, 26.724810345780593],
    [-96.42355704777482, 26.216515704595633],
    [-80.68508661702214, 24.546812350183075],
    [-75.56173032587596, 26.814533788629998],
    [-67.1540159827795, 44.40095539443753],
    [-68.07548734644243, 46.981170472447374],
    [-69.17500995805074, 46.98158998130476],
    [-70.7598785138901, 44.87172183866657],
    [-74.84994741250935, 44.748084983808],
    [-77.62168256782745, 43.005725611950055],
    [-82.45987924104175, 41.41068867019324],
    [-83.38318501671864, 42.09979904377044],
    [-82.5905167831457, 45.06163491639556],
    [-84.83301910769038, 46.83552648258547],
    [-88.26350848510909, 48.143646480291835],
    [-90.06706251069104, 47.553445811024204],
    [-95.03745451438925, 48.9881557770297],
    [-98.45773319567587, 48.94699366043251],
    [-101.7018751401119, 48.98284560308372],
    [-108.43164852530356, 48.81973606668503],
    [-115.07339190755627, 48.93699058308441],
    [-121.82530604190744, 48.9830983403776],
    [-122.22085227110232, 48.63535795404536],
    [-124.59504332589562, 47.695726563030405],
    [-125.1803892906456, 35.26328285844432]
]

def create_usa_geometry():
    """Create an Earth Engine geometry object for the contiguous USA."""
    return ee.Geometry.Polygon([usa_coords])

def compute_area(feature):
    """Compute the area of a feature and set it as a property."""
    return feature.set({'area': feature.area()})

def compute_centroid(feature):
    """Compute the centroid coordinates of a feature and set them as properties."""
    centroid = feature.geometry().centroid().coordinates()
    return feature.set({
        'lon': centroid.get(0),
        'lat': centroid.get(1)
    })

def ee_featurecollection_to_gdf(fc):
    """Convert Earth Engine FeatureCollection to GeoPandas DataFrame."""
    features = fc.getInfo()['features']
    
    # Extract the geometry and properties from each feature
    geometries = []
    properties = []
    
    for feature in features:
        # Convert GEE geometry to Shapely geometry
        geom = feature['geometry']
        if geom['type'] == 'Polygon':
            geometry = Polygon(geom['coordinates'][0])
        else:
            # Handle other geometry types if needed
            continue
            
        geometries.append(geometry)
        properties.append(feature['properties'])
    
    # Create GeoDataFrame
    df = pd.DataFrame(properties)
    gdf = gpd.GeoDataFrame(df, geometry=geometries, crs="EPSG:4326")
    
    # Convert area to numeric
    if 'area' in gdf.columns:
        gdf['area'] = pd.to_numeric(gdf['area'])
    
    return gdf

def get_daily_fires(year, min_size=1e7, region=None):
    """
    Get daily fire perimeters from the GlobFire database for a specific year and minimum size.
    
    Args:
        year (str): The year to get fires for
        min_size (float): Minimum fire size in square meters (default: 1e7)
        region (ee.Geometry, optional): Region to filter fires. Defaults to contiguous USA.
    
    Returns:
        geopandas.GeoDataFrame: GeoDataFrame containing daily fire perimeter data
    """
    # Set up region
    if region is None:
        region = create_usa_geometry()
    
    # Create collection name for the specified year
    collection_name = f'JRC/GWIS/GlobFire/v2/DailyPerimeters/{year}'
    
    try:
        # Get and filter fire polygons
        polygons = (ee.FeatureCollection(collection_name)
                   .filterBounds(region))
        
        # Apply area calculations and filters
        polygons = polygons.map(compute_area)
        polygons = (polygons
                   .filter(ee.Filter.gt('area', min_size))
                   .filter(ee.Filter.lt('area', 1e20)))
        
        # Compute additional properties
        polygons = polygons.map(compute_centroid)
        
        # Convert to GeoDataFrame
        gdf = ee_featurecollection_to_gdf(polygons)
        
        # Add date column if not present
        if 'date' not in gdf.columns:
            gdf['date'] = pd.to_datetime(gdf['IDate'], unit='ms')
        
        return gdf
        
    except ee.ee_exception.EEException as e:
        print(f"Error accessing collection for year {year}: {str(e)}")
        print("Note: Daily perimeters might not be available for this year.")
        return None

def analyze_daily_fires(gdf):
    """
    Perform basic analysis on the daily fire perimeters.
    
    Args:
        gdf (geopandas.GeoDataFrame): GeoDataFrame containing fire data
    
    Returns:
        dict: Dictionary containing analysis results
    """
    if gdf is None or len(gdf) == 0:
        return None
        
    analysis = {
        'total_fires': len(gdf),
        'total_area_km2': gdf['area'].sum() / 1e6,  # Convert to km²
        'mean_area_km2': gdf['area'].mean() / 1e6,
        'max_area_km2': gdf['area'].max() / 1e6,
        'date_range': f"{gdf['IDate'].min()} to {gdf['IDate'].max()}"
    }
    
    if 'fid' in gdf.columns:
        analysis['unique_fires'] = gdf['fid'].nunique()
    
    return analysis

if __name__ == "__main__":
    # Example usage
    YEAR = "2020"
    MIN_SIZE = 1e7  # 10 square kilometers
    
    # Get daily fire perimeters as a GeoDataFrame
    fires_gdf = get_daily_fires(YEAR, MIN_SIZE)
    
    if fires_gdf is not None:
        # Perform basic analysis
        analysis_results = analyze_daily_fires(fires_gdf)
        
        print(f"\nAnalysis Results for {YEAR}:")
        for key, value in analysis_results.items():
            print(f"{key}: {value}")
        
        # # Example: Save to file
        # output_file = f"us_daily_fires_{YEAR}.gpkg"
        # fires_gdf.to_file(output_file, driver="GPKG")
        # print(f"\nSaved to {output_file}")
        
        # Example: Show temporal distribution
        print("\nFires by month:")
        monthly_counts = fires_gdf.groupby(fires_gdf['date'].dt.month).size()
        print(monthly_counts)

In [ ]:
fires_gdf.head()

In [ ]:
for i in range(len(fires_gdf[fires_gdf['Id'] == 24332628])):
    fires_gdf[fires_gdf['Id'] == 24332628].sort_values('IDate').iloc[i].geometry.plot()

In [ ]:
# drop everything that does not have at least 2 Id in fires_gdf
fires_gdf_reduced = fires_gdf[fires_gdf['Id'].isin(fires_gdf['Id'].value_counts()[fires_gdf['Id'].value_counts() > 1].index)]# save to geojson
fires_gdf_reduced.to_file(f"data/perims/combined_fires_{YEAR}.geojson", driver="GeoJSON")

### Download daily and final perims

In [ ]:
import ee
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, mapping
import datetime

# Initialize the Earth Engine API
ee.Initialize()

# Define the geometry for contiguous USA
usa_coords = [
    [-125.1803892906456, 35.26328285844432],
    [-117.08916345892665, 33.2311514593429],
    [-114.35640058749676, 32.92199940444295],
    [-110.88773544819885, 31.612036247094473],
    [-108.91086200144109, 31.7082477979397],
    [-106.80030780089378, 32.42079476218232],
    [-103.63413436750255, 29.786401496314422],
    [-101.87558377066483, 30.622527701868453],
    [-99.40039768482492, 28.04018292597704],
    [-98.69085295525215, 26.724810345780593],
    [-96.42355704777482, 26.216515704595633],
    [-80.68508661702214, 24.546812350183075],
    [-75.56173032587596, 26.814533788629998],
    [-67.1540159827795, 44.40095539443753],
    [-68.07548734644243, 46.981170472447374],
    [-69.17500995805074, 46.98158998130476],
    [-70.7598785138901, 44.87172183866657],
    [-74.84994741250935, 44.748084983808],
    [-77.62168256782745, 43.005725611950055],
    [-82.45987924104175, 41.41068867019324],
    [-83.38318501671864, 42.09979904377044],
    [-82.5905167831457, 45.06163491639556],
    [-84.83301910769038, 46.83552648258547],
    [-88.26350848510909, 48.143646480291835],
    [-90.06706251069104, 47.553445811024204],
    [-95.03745451438925, 48.9881557770297],
    [-98.45773319567587, 48.94699366043251],
    [-101.7018751401119, 48.98284560308372],
    [-108.43164852530356, 48.81973606668503],
    [-115.07339190755627, 48.93699058308441],
    [-121.82530604190744, 48.9830983403776],
    [-122.22085227110232, 48.63535795404536],
    [-124.59504332589562, 47.695726563030405],
    [-125.1803892906456, 35.26328285844432]
]

def create_usa_geometry():
    """Create an Earth Engine geometry object for the contiguous USA."""
    return ee.Geometry.Polygon([usa_coords])

def compute_area(feature):
    """Compute the area of a feature and set it as a property."""
    return feature.set({'area': feature.area()})

def compute_centroid(feature):
    """Compute the centroid coordinates of a feature and set them as properties."""
    centroid = feature.geometry().centroid().coordinates()
    return feature.set({
        'lon': centroid.get(0),
        'lat': centroid.get(1)
    })

def ee_featurecollection_to_gdf(fc):
    """Convert Earth Engine FeatureCollection to GeoPandas DataFrame."""
    features = fc.getInfo()['features']
    
    # Extract the geometry and properties from each feature
    geometries = []
    properties = []
    
    for feature in features:
        # Convert GEE geometry to Shapely geometry
        geom = feature['geometry']
        if geom['type'] == 'Polygon':
            geometry = Polygon(geom['coordinates'][0])
        else:
            # Handle other geometry types if needed
            continue
            
        geometries.append(geometry)
        properties.append(feature['properties'])
    
    # Create GeoDataFrame
    df = pd.DataFrame(properties)
    gdf = gpd.GeoDataFrame(df, geometry=geometries, crs="EPSG:4326")
    
    # Convert area to numeric
    if 'area' in gdf.columns:
        gdf['area'] = pd.to_numeric(gdf['area'])
    
    return gdf

def get_daily_fires(year, min_size=1e7, region=None):
    """
    Get daily fire perimeters from the GlobFire database.
    
    Args:
        year (str): The year to get fires for
        min_size (float): Minimum fire size in square meters
        region (ee.Geometry, optional): Region to filter fires
    """
    if region is None:
        region = create_usa_geometry()
    
    collection_name = f'JRC/GWIS/GlobFire/v2/DailyPerimeters/{year}'
    
    try:
        polygons = (ee.FeatureCollection(collection_name)
                   .filterBounds(region))
        
        polygons = polygons.map(compute_area)
        polygons = (polygons
                   .filter(ee.Filter.gt('area', min_size))
                   .filter(ee.Filter.lt('area', 1e20)))
        
        polygons = polygons.map(compute_centroid)
        
        gdf = ee_featurecollection_to_gdf(polygons)
        
        if not gdf.empty:
            gdf['source'] = 'daily'
            # Convert IDate to datetime directly for each row
            gdf['date'] = pd.to_datetime(gdf['IDate'], unit='ms')
            # For daily perimeters, end_date is same as start date
            gdf['end_date'] = gdf['date']
        
        return gdf
        
    except ee.ee_exception.EEException as e:
        print(f"Error accessing daily collection for {year}: {str(e)}")
        return None

def get_final_fires(year, min_size=1e7, region=None):
    """
    Get final fire perimeters from the GlobFire database.
    
    Args:
        year (str): The year to get fires for
        min_size (float): Minimum fire size in square meters
        region (ee.Geometry, optional): Region to filter fires
    """
    if region is None:
        region = create_usa_geometry()
    
    start_date = ee.Date(f'{year}-01-01')
    end_date = ee.Date(f'{year}-12-31')
    
    try:
        polygons = (ee.FeatureCollection('JRC/GWIS/GlobFire/v2/FinalPerimeters')
                   .filter(ee.Filter.gt('IDate', start_date.millis()))
                   .filter(ee.Filter.lt('IDate', end_date.millis()))
                   .filterBounds(region))
        
        polygons = polygons.map(compute_area)
        polygons = (polygons
                   .filter(ee.Filter.gt('area', min_size))
                   .filter(ee.Filter.lt('area', 1e20)))
        
        polygons = polygons.map(compute_centroid)
        
        gdf = ee_featurecollection_to_gdf(polygons)
        
        if not gdf.empty:
            gdf['source'] = 'final'
            # Convert IDate and FDate to datetime for each row
            gdf['date'] = pd.to_datetime(gdf['IDate'], unit='ms')
            gdf['end_date'] = pd.to_datetime(gdf['FDate'], unit='ms')
        
        return gdf
        
    except ee.ee_exception.EEException as e:
        print(f"Error accessing final perimeters for {year}: {str(e)}")
        return None

def get_combined_fires(year, min_size=1e7, region=None):
    """
    Get both daily and final fire perimeters and combine them based on Id.
    
    Args:
        year (str): The year to get fires for
        min_size (float): Minimum fire size in square meters
        region (ee.Geometry, optional): Region to filter fires
    
    Returns:
        tuple: (combined_gdf, daily_gdf, final_gdf)
    """
    daily_gdf = get_daily_fires(year, min_size, region)
    final_gdf = get_final_fires(year, min_size, region)
    
    if daily_gdf is None and final_gdf is None:
        return None, None, None
    
    # Ensure we have dataframes to work with
    if daily_gdf is None:
        daily_gdf = gpd.GeoDataFrame()
    if final_gdf is None:
        final_gdf = gpd.GeoDataFrame()
    
    # Convert timestamps consistently
    for gdf in [daily_gdf, final_gdf]:
        if not gdf.empty:
            # Convert all timestamp fields to numeric if they aren't already
            for col in ['IDate', 'FDate']:
                if col in gdf.columns:
                    gdf[col] = pd.to_numeric(gdf[col])
            for col in ['FDate']:
                if col in gdf.columns:
                    gdf[col] = gdf['end_date']
    
    # Get unique fire IDs
    all_ids = pd.concat([
        daily_gdf['Id'] if not daily_gdf.empty else pd.Series(dtype=int),
        final_gdf['Id'] if not final_gdf.empty else pd.Series(dtype=int)
    ]).unique()
    
    combined_data = []
    
    for fire_id in all_ids:
        # Get daily perimeters for this fire
        daily_fire = daily_gdf[daily_gdf['Id'] == fire_id] if not daily_gdf.empty else None
        # Get final perimeter for this fire
        final_fire = final_gdf[final_gdf['Id'] == fire_id] if not final_gdf.empty else None
        
        if daily_fire is not None and not daily_fire.empty:
            # Add all daily perimeters
            combined_data.append(daily_fire)
        
        if final_fire is not None and not final_fire.empty:
            # Add final perimeter
            combined_data.append(final_fire)
    
    if not combined_data:
        return None, None, None
    
    # Combine all data
    combined_gdf = pd.concat(combined_data, ignore_index=True)
    
    # Sort by Id and date for consistency
    combined_gdf = combined_gdf.sort_values(['Id', 'date'])
    
    return combined_gdf, daily_gdf, final_gdf

def analyze_fires(gdf):
    """
    Perform basic analysis on fire perimeters.
    """
    if gdf is None or len(gdf) == 0:
        return None
    
    # Basic statistics
    stats = {
        'total_fires': len(gdf),
        'unique_fires': gdf['Id'].nunique(),
        'total_area_km2': gdf['area'].sum() / 1e6,
        'mean_area_km2': gdf['area'].mean() / 1e6,
        'max_area_km2': gdf['area'].max() / 1e6,
        'date_range': f"{gdf['date'].min()} to {gdf['end_date'].max()}"
    }
    
    # Add source-specific counts
    if 'source' in gdf.columns:
        source_counts = gdf['source'].value_counts()
        for source in source_counts.index:
            stats[f'{source}_fires'] = source_counts[source]
            
        # Add counts of fires with both daily and final perimeters
        fires_with_both = (gdf.groupby('Id')['source']
                          .nunique()
                          .where(lambda x: x > 1)
                          .count())
        stats['fires_with_both_perims'] = fires_with_both
    
    return stats

# Example usage
YEAR = "2020"
MIN_SIZE = 1e7  # 10 square kilometers

# Get both daily and final perimeters
combined_gdf, daily_gdf, final_gdf = get_combined_fires(YEAR, MIN_SIZE)

if combined_gdf is not None:
    print(f"\nAnalysis Results for {YEAR}:")
    
    print("\nCombined Perimeters:")
    combined_stats = analyze_fires(combined_gdf)
    for key, value in combined_stats.items():
        print(f"{key}: {value}")
    
    if daily_gdf is not None:
        print("\nDaily Perimeters:")
        daily_stats = analyze_fires(daily_gdf)
        for key, value in daily_stats.items():
            print(f"{key}: {value}")
    
    if final_gdf is not None:
        print("\nFinal Perimeters:")
        final_stats = analyze_fires(final_gdf)
        for key, value in final_stats.items():
            print(f"{key}: {value}")
    
    # Temporal distribution
    print("\nFires by month:")
    monthly_counts = combined_gdf.groupby([combined_gdf['date'].dt.month, 'source']).size().unstack(fill_value=0)
    print(monthly_counts)

# drop everything that does not have at least 2 Id in combined_gdf
combined_gdf_reduced = combined_gdf[combined_gdf['Id'].isin(combined_gdf['Id'].value_counts()[combined_gdf['Id'].value_counts() > 1].index)]# save to geojson
combined_gdf_reduced.to_file(f"data/perims/combined_fires_{YEAR}.geojson", driver="GeoJSON")

In [ ]:
def get_matching_perimeters(daily_gdf, final_gdf):
    """
    Create a dataframe containing only fire perimeters that appear in both daily and final datasets.
    
    Args:
        daily_gdf (GeoDataFrame): DataFrame containing daily fire perimeters
        final_gdf (GeoDataFrame): DataFrame containing final fire perimeters
    
    Returns:
        GeoDataFrame: Combined DataFrame containing only fires that appear in both datasets
    """
    # Handle cases where either dataframe is None or empty
    if daily_gdf is None or final_gdf is None or daily_gdf.empty or final_gdf.empty:
        return None
    
    # Get IDs that appear in both datasets
    daily_ids = set(daily_gdf['Id'].unique())
    final_ids = set(final_gdf['Id'].unique())
    matching_ids = daily_ids.intersection(final_ids)
    
    if not matching_ids:
        return None
    
    # Filter both dataframes for matching IDs
    daily_matches = daily_gdf[daily_gdf['Id'].isin(matching_ids)].copy()
    final_matches = final_gdf[final_gdf['Id'].isin(matching_ids)].copy()
    
    # Combine the filtered dataframes
    matching_perims = pd.concat([daily_matches, final_matches], ignore_index=True)
    
    # Sort by Id and date for consistency
    matching_perims = matching_perims.sort_values(['Id', 'date'])
    
    # Add some useful metadata
    matching_perims['match_type'] = 'both_sources'
    
    return matching_perims

# Example usage in main script:
if __name__ == "__main__":
    YEAR = "2020"
    MIN_SIZE = 1e7  # 10 square kilometers
    
    # Get fire perimeters
    # combined_gdf, daily_gdf, final_gdf = get_combined_fires(YEAR, MIN_SIZE)
    
    # Get matching perimeters
    matching_perims = get_matching_perimeters(daily_gdf, final_gdf)
    
    if matching_perims is not None:
        print("\nMatching Perimeters Analysis:")
        matching_stats = analyze_fires(matching_perims)
        for key, value in matching_stats.items():
            print(f"{key}: {value}")
            
        # Additional statistics specific to matching perimeters
        print("\nDetailed Matching Statistics:")
        unique_matches = matching_perims['Id'].nunique()
        print(f"Number of unique fires with both daily and final perimeters: {unique_matches}")
        
        # Calculate average number of daily perims per fire
        daily_counts = matching_perims[matching_perims['source'] == 'daily'].groupby('Id').size()
        print(f"Average daily perimeters per fire: {daily_counts.mean():.1f}")
        print(f"Max daily perimeters for a single fire: {daily_counts.max()}")
        
        # Show distribution of time spans
        time_spans = matching_perims.groupby('Id').agg({
            'date': 'min',
            'end_date': 'max'
        })
        time_spans['duration_days'] = (time_spans['end_date'] - time_spans['date']).dt.total_seconds() / (24*3600)
        print(f"\nFire duration statistics (days):")
        print(f"Mean duration: {time_spans['duration_days'].mean():.1f}")
        print(f"Median duration: {time_spans['duration_days'].median():.1f}")
        print(f"Max duration: {time_spans['duration_days'].max():.1f}")
    else:
        print("\nNo matching perimeters found between daily and final datasets.")

In [ ]:
combined_gdf[combined_gdf.Id == 25294714]

In [ ]:
# show Ids that match between daily and final as keys in a dict, with the number of times they appear in daily source as values
fires = {}
for i in final_gdf['Id']:
    if i in daily_gdf['Id'].values:
        fires[i] = daily_gdf[daily_gdf['Id'] == i].shape[0]

fires

### Visualize the perims

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
import pandas as pd
from shapely.ops import unary_union
from shapely.geometry import MultiPolygon, Polygon

def calculate_area_km2(geometry, lat, lon):
    """
    Calculate area in square kilometers using an equal area projection.
    
    Args:
        geometry: Shapely geometry
        lat: Latitude of centroid for projection center
        lon: Longitude of centroid for projection center
    
    Returns:
        float: Area in square kilometers
    """
    # Create a temporary GeoDataFrame with the geometry
    temp_gdf = gpd.GeoDataFrame(geometry=[geometry], crs="EPSG:4326")
    
    # Project to an equal area projection centered on the area of interest
    proj_string = f"+proj=aea +lat_1={lat-5} +lat_2={lat+5} +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    temp_gdf = temp_gdf.to_crs(proj_string)
    
    # Calculate area in km²
    return temp_gdf.area.iloc[0] / 1e6

def get_polygon_coords(geometry):
    """
    Extract coordinates from either a Polygon or MultiPolygon.
    Returns a list of coordinate arrays, one per ring.
    """
    coords_list = []
    
    if isinstance(geometry, MultiPolygon):
        for polygon in geometry.geoms:
            coords_list.append(np.array(polygon.exterior.coords))
            for interior in polygon.interiors:
                coords_list.append(np.array(interior.coords))
    elif isinstance(geometry, Polygon):
        coords_list.append(np.array(geometry.exterior.coords))
        for interior in geometry.interiors:
            coords_list.append(np.array(interior.coords))
    
    return coords_list

def animate_fire_progression(gdf, fire_id, interval=500):
    """
    Create an animated visualization of cumulative fire progression.
    """
    # Filter for the specific fire and sort by date
    fire_perimeters = gdf[gdf['Id'] == fire_id].sort_values('IDate')
    
    # Pre-compute cumulative geometries
    cumulative_geometries = []
    current_geometry = None
    
    for idx, row in fire_perimeters.iterrows():
        if current_geometry is None:
            current_geometry = row.geometry
        else:
            current_geometry = unary_union([current_geometry, row.geometry])
        cumulative_geometries.append(current_geometry)
    
    # Get centroid of final perimeter for area calculation
    final_centroid = cumulative_geometries[-1].centroid
    
    # Create GeoDataFrame with cumulative geometries
    cumulative_gdf = gpd.GeoDataFrame(
        {
            'geometry': cumulative_geometries,
            'IDate': fire_perimeters['IDate'].values
        },
        crs="EPSG:4326"
    )
    
    # Calculate areas using the centroid-based projection
    cumulative_gdf['area_km2'] = [
        calculate_area_km2(geom, final_centroid.y, final_centroid.x)
        for geom in cumulative_gdf.geometry
    ]
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Set consistent plot limits
    total_bounds = cumulative_gdf.total_bounds
    padding = 0.1
    width = total_bounds[2] - total_bounds[0]
    height = total_bounds[3] - total_bounds[1]
    ax.set_xlim(total_bounds[0] - width * padding, 
                total_bounds[2] + width * padding)
    ax.set_ylim(total_bounds[1] - height * padding, 
                total_bounds[3] + height * padding)
    
    # Create lists to store all lines and fills
    lines = []
    fills = []
    
    # Initialize with empty data
    for i in range(10):
        line, = ax.plot([], [], 'r-', lw=2)
        fill = ax.fill([], [], 'r', alpha=0.3)[0]
        lines.append(line)
        fills.append(fill)
    
    area_text = ax.text(0.02, 0.98, '', transform=ax.transAxes, 
                       verticalalignment='top')
    title = ax.set_title('')
    
    # Format dates for display
    dates = pd.to_datetime(cumulative_gdf['IDate'], unit='ms')
    date_strings = dates.dt.strftime('%Y-%m-%d')
    
    def init():
        for line, fill in zip(lines, fills):
            line.set_data([], [])
            fill.set_xy(np.empty((0, 2)))
        area_text.set_text('')
        return lines + fills + [area_text, title]
    
    def animate(frame):
        perimeter = cumulative_gdf.iloc[frame]
        all_coords = get_polygon_coords(perimeter.geometry)
        
        for i, coords in enumerate(all_coords):
            if i < len(lines):
                lines[i].set_data(coords[:, 0], coords[:, 1])
                fills[i].set_xy(coords)
            else:
                line, = ax.plot([], [], 'r-', lw=2)
                fill = ax.fill([], [], 'r', alpha=0.3)[0]
                lines.append(line)
                fills.append(fill)
                lines[i].set_data(coords[:, 0], coords[:, 1])
                fills[i].set_xy(coords)
        
        for i in range(len(all_coords), len(lines)):
            lines[i].set_data([], [])
            fills[i].set_xy(np.empty((0, 2)))
        
        area_text.set_text(f'Area: {perimeter.area_km2:.2f} km²')
        title.set_text(f'Fire ID {fire_id} - {date_strings.iloc[frame]}')
        
        return lines + fills + [area_text, title]
    
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                 frames=len(cumulative_gdf),
                                 interval=interval, blit=True)
    
    html = anim.to_jshtml()
    plt.close()
    
    return HTML(html)

def create_fire_summary(gdf, fire_id):
    """
    Create a summary of cumulative fire progression.
    """
    fire_perimeters = gdf[gdf['Id'] == fire_id].sort_values('IDate')
    
    # Calculate cumulative geometries
    cumulative_geometries = []
    current_geometry = None
    
    for idx, row in fire_perimeters.iterrows():
        if current_geometry is None:
            current_geometry = row.geometry
        else:
            current_geometry = unary_union([current_geometry, row.geometry])
        cumulative_geometries.append(current_geometry)
    
    # Get centroid of final perimeter for area calculation
    final_centroid = cumulative_geometries[-1].centroid
    
    # Calculate areas
    areas = [
        calculate_area_km2(geom, final_centroid.y, final_centroid.x)
        for geom in cumulative_geometries
    ]
    
    # Convert dates
    dates = pd.to_datetime(fire_perimeters['IDate'], unit='ms')
    
    summary = {
        'start_date': dates.min(),
        'end_date': dates.max(),
        'duration_days': (dates.max() - dates.min()).days,
        'num_updates': len(fire_perimeters),
        'initial_area_km2': areas[0],
        'final_area_km2': areas[-1],
        'total_growth_km2': areas[-1] - areas[0],
        'average_daily_growth_km2': (areas[-1] - areas[0]) / max((dates.max() - dates.min()).days, 1)
    }
    
    return summary

# Create and display the animation
animation = animate_fire_progression(fires_gdf, 24332801)
display(animation)

# Get summary statistics
summary = create_fire_summary(fires_gdf, 24332801)
print("\nFire Summary:")
for key, value in summary.items():
    print(f"{key}: {value}")

#### Visualize all

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
import pandas as pd
from shapely.ops import unary_union
from shapely.geometry import MultiPolygon, Polygon

def calculate_area_km2(geometry, lat, lon):
    """
    Calculate area in square kilometers using an equal area projection.
    """
    temp_gdf = gpd.GeoDataFrame(geometry=[geometry], crs="EPSG:4326")
    proj_string = f"+proj=aea +lat_1={lat-5} +lat_2={lat+5} +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    temp_gdf = temp_gdf.to_crs(proj_string)
    return temp_gdf.area.iloc[0] / 1e6

def get_polygon_coords(geometry):
    """
    Extract coordinates from either a Polygon or MultiPolygon.
    """
    coords_list = []
    if isinstance(geometry, MultiPolygon):
        for polygon in geometry.geoms:
            coords_list.append(np.array(polygon.exterior.coords))
            for interior in polygon.interiors:
                coords_list.append(np.array(interior.coords))
    elif isinstance(geometry, Polygon):
        coords_list.append(np.array(geometry.exterior.coords))
        for interior in geometry.interiors:
            coords_list.append(np.array(interior.coords))
    return coords_list

def animate_fire_progression(gdf, fire_id, interval=500):
    """
    Create an animated visualization of fire progression with final perimeter.
    """
    # Split daily and final perimeters
    daily_perims = gdf[(gdf['Id'] == fire_id) & (gdf['source'] == 'daily')].sort_values('date')
    final_perim = gdf[(gdf['Id'] == fire_id) & (gdf['source'] == 'final')]
    
    # Pre-compute daily cumulative geometries
    cumulative_geometries = []
    current_geometry = None
    
    for idx, row in daily_perims.iterrows():
        if current_geometry is None:
            current_geometry = row.geometry
        else:
            current_geometry = unary_union([current_geometry, row.geometry])
        cumulative_geometries.append(current_geometry)
    
    # Add final perimeter as the last frame if it exists
    if not final_perim.empty:
        cumulative_geometries.append(final_perim.iloc[0].geometry)
        # For daily perimeters, use 'date', for final perimeter use 'end_date'
        dates = pd.concat([
            daily_perims['date'],
            pd.Series([final_perim.iloc[0]['end_date']])
        ]).reset_index(drop=True)
    else:
        dates = daily_perims['date'].reset_index(drop=True)
    
    # Get centroid of final perimeter for area calculation
    final_centroid = cumulative_geometries[-1].centroid
    
    # Create GeoDataFrame with cumulative geometries
    cumulative_gdf = gpd.GeoDataFrame(
        {
            'geometry': cumulative_geometries,
            'date': dates,
            'is_final': [False] * len(daily_perims) + [True] * (1 if not final_perim.empty else 0)
        },
        crs="EPSG:4326"
    )
    
    # Calculate areas
    cumulative_gdf['area_km2'] = [
        calculate_area_km2(geom, final_centroid.y, final_centroid.x)
        for geom in cumulative_gdf.geometry
    ]
    
    # Create figure and axis
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Set consistent plot limits
    total_bounds = cumulative_gdf.total_bounds
    padding = 0.1
    width = total_bounds[2] - total_bounds[0]
    height = total_bounds[3] - total_bounds[1]
    ax.set_xlim(total_bounds[0] - width * padding, 
                total_bounds[2] + width * padding)
    ax.set_ylim(total_bounds[1] - height * padding, 
                total_bounds[3] + height * padding)
    
    # Create lists to store all lines and fills
    lines = []
    fills = []
    
    # Initialize with empty data
    for i in range(10):
        line, = ax.plot([], [], 'r-', lw=2)
        fill = ax.fill([], [], 'r', alpha=0.3)[0]
        lines.append(line)
        fills.append(fill)
    
    area_text = ax.text(0.02, 0.98, '', transform=ax.transAxes, 
                       verticalalignment='top')
    title = ax.set_title('')
    
    def init():
        for line, fill in zip(lines, fills):
            line.set_data([], [])
            fill.set_xy(np.empty((0, 2)))
        area_text.set_text('')
        return lines + fills + [area_text, title]
    
    def animate(frame):
        perimeter = cumulative_gdf.iloc[frame]
        all_coords = get_polygon_coords(perimeter.geometry)
        
        # Use different colors for final perimeter
        color = 'blue' if perimeter.is_final else 'r'
        alpha = 0.5 if perimeter.is_final else 0.3
        
        for i, coords in enumerate(all_coords):
            if i < len(lines):
                lines[i].set_color(color)
                lines[i].set_data(coords[:, 0], coords[:, 1])
                fills[i].set_facecolor(color)
                fills[i].set_alpha(alpha)
                fills[i].set_xy(coords)
            else:
                line, = ax.plot([], [], f'{color}-', lw=2)
                fill = ax.fill([], [], color, alpha=alpha)[0]
                lines.append(line)
                fills.append(fill)
                lines[i].set_data(coords[:, 0], coords[:, 1])
                fills[i].set_xy(coords)
        
        for i in range(len(all_coords), len(lines)):
            lines[i].set_data([], [])
            fills[i].set_xy(np.empty((0, 2)))
        
        # Update area text
        area_text.set_text(f'Area: {perimeter.area_km2:.2f} km²')
        
        # Update title with date and type
        perim_type = "Final Perimeter" if perimeter.is_final else "Daily Perimeter"
        display_date = perimeter.date.strftime("%Y-%m-%d")
        title.set_text(f'Fire ID {fire_id} - {display_date} ({perim_type})')
        
        return lines + fills + [area_text, title]
    
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                 frames=len(cumulative_gdf),
                                 interval=interval, blit=True)
    
    html = anim.to_jshtml()
    plt.close()
    
    return HTML(html)

def create_fire_summary(gdf, fire_id):
    """
    Create a summary of fire progression including both daily and final perimeters.
    """
    # Get daily perimeters and optional final perimeter
    daily_perims = gdf[(gdf['Id'] == fire_id) & (gdf['source'] == 'daily')].sort_values('date')
    final_perim = gdf[(gdf['Id'] == fire_id) & (gdf['source'] == 'final')]
    
    # Calculate cumulative geometries for daily progression
    cumulative_geometries = []
    current_geometry = None
    
    for idx, row in daily_perims.iterrows():
        if current_geometry is None:
            current_geometry = row.geometry
        else:
            current_geometry = unary_union([current_geometry, row.geometry])
        cumulative_geometries.append(current_geometry)
    
    # Add final perimeter if available
    if not final_perim.empty:
        final_geometry = final_perim.iloc[0].geometry
        final_date = final_perim.iloc[0].end_date
    else:
        final_geometry = cumulative_geometries[-1] if cumulative_geometries else None
        final_date = daily_perims.iloc[-1].end_date if not daily_perims.empty else None
    
    if not cumulative_geometries and final_geometry is None:
        return None
    
    # Get centroid for area calculations
    centroid = final_geometry.centroid if final_geometry else cumulative_geometries[-1].centroid
    
    # Calculate areas
    daily_areas = [
        calculate_area_km2(geom, centroid.y, centroid.x)
        for geom in cumulative_geometries
    ]
    
    if final_geometry:
        final_area = calculate_area_km2(final_geometry, centroid.y, centroid.x)
    else:
        final_area = daily_areas[-1] if daily_areas else None
    
    # Get dates
    start_date = daily_perims.iloc[0].end_date if not daily_perims.empty else final_date
    
    summary = {
        'start_date': start_date,
        'end_date': final_date,
        'duration_days': (final_date - start_date).days,
        'num_daily_updates': len(daily_perims),
        'has_final_perimeter': not final_perim.empty,
        'initial_area_km2': daily_areas[0] if daily_areas else final_area,
        'final_area_km2': final_area,
        'total_growth_km2': final_area - daily_areas[0] if daily_areas else 0,
        'average_daily_growth_km2': (final_area - daily_areas[0]) / max((final_date - start_date).days, 1) if daily_areas else 0
    }
    
    return summary

In [ ]:
# Create and display the animation
# animation = None
animation = animate_fire_progression(combined_gdf, 20778153)
display(animation)

# Get summary statistics
summary = create_fire_summary(combined_gdf, 20778153)
print("\nFire Summary:")
for key, value in summary.items():
    print(f"{key}: {value}")

In [ ]:

# drop everything that does not have at least 2 Id in combined_gdf
combined_gdf_reduced = combined_gdf[combined_gdf['Id'].isin(combined_gdf['Id'].value_counts()[combined_gdf['Id'].value_counts() > 1].index)]# save to geojson
combined_gdf_reduced.to_file(f"data/perims/combined_fires_{YEAR}.geojson", driver="GeoJSON")